# Palo Alto data merge

#### 1. Chargerpoint data merge
Get names of all xlsx files in folder

In [1]:
import pandas as pd
import glob
import numpy as np

In [2]:
path =r'/home/ar/Google/DTU/Courses/Advanced business analytics/ABA - Project /Datasets/Palo_Alto'#path of stored xlsx files
filenames = glob.glob(path + "/*.xlsx")
print(filenames)

['/home/ar/Google/DTU/Courses/Advanced business analytics/ABA - Project /Datasets/Palo_Alto/CY21Q3 EV Charging Event Data.xlsx', '/home/ar/Google/DTU/Courses/Advanced business analytics/ABA - Project /Datasets/Palo_Alto/CY23Q2 EV Charging Event Data.xlsx', '/home/ar/Google/DTU/Courses/Advanced business analytics/ABA - Project /Datasets/Palo_Alto/CY21Q1 EV Charging Event Data.xlsx', '/home/ar/Google/DTU/Courses/Advanced business analytics/ABA - Project /Datasets/Palo_Alto/CY23Q4 EV Charging Event Data.xlsx', '/home/ar/Google/DTU/Courses/Advanced business analytics/ABA - Project /Datasets/Palo_Alto/CY21Q4 EV Charging Event Data.xlsx', '/home/ar/Google/DTU/Courses/Advanced business analytics/ABA - Project /Datasets/Palo_Alto/CY23Q1 EV Charging Event Data.xlsx', '/home/ar/Google/DTU/Courses/Advanced business analytics/ABA - Project /Datasets/Palo_Alto/CY22Q4 EV Charging Event Data.xlsx', '/home/ar/Google/DTU/Courses/Advanced business analytics/ABA - Project /Datasets/Palo_Alto/CY22Q3 EV Ch

Initiallize empty dataframe and concatenate all files from 2021 onwards, store into a new csv

In [3]:
df = pd.DataFrame()

for i in filenames:
    data = pd.read_excel(i)
    df = pd.concat([df, data])

# Create a csv file with only chargerpoint chargers from 2021 to 2023
df.to_csv('updated_till_2024_chargerpoint', index=False)  # Set index=False to exclude the index column

Read the csv files corresponding to before 2021 and after, mege both csv files

In [4]:
before = pd.read_csv('/home/ar/Google/DTU/Courses/Advanced business analytics/ABA - Project /Datasets/Palo_Alto/EVChargingStationUsage.csv') # Path to old palo alto dataset
after = pd.read_csv('/home/ar/Google/DTU/Courses/Advanced business analytics/ABA - Project /Datasets/Palo_Alto/updated_till_2024_chargerpoint')# Path to concatenated chargerpoint paloalto dataset in previous cell

together=pd.DataFrame()

together = pd.concat([before, after])
# Create a csv file with only chargerpoint chargers from 2011 to 2023
together.to_csv('updated_till_24_only_charger_point', index=False)

/tmp/ipykernel_15998/3669125881.py:1: DtypeWarning: Columns (29,30,32) have mixed types. Specify dtype option on import or set low_memory=False.
  before = pd.read_csv('/home/ar/Google/DTU/Courses/Advanced business analytics/ABA - Project /Datasets/Palo_Alto/EVChargingStationUsage.csv') # Path to old palo alto dataset
/tmp/ipykernel_15998/3669125881.py:2: DtypeWarning: Columns (30,31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  after = pd.read_csv('/home/ar/Google/DTU/Courses/Advanced business analytics/ABA - Project /Datasets/Palo_Alto/updated_till_2024_chargerpoint')# Path to concatenated chargerpoint paloalto dataset in previous cell


#### 2. Flexpoint data reformatting
Get the list of all files  from 22 onwards which contain the new types of chargers with new format

In [5]:
new_chargers=[]
for file in filenames:
    if '22' in file or '23' in file:
        new_chargers.append(file)
    else:
        continue
print(new_chargers)

['/home/ar/Google/DTU/Courses/Advanced business analytics/ABA - Project /Datasets/Palo_Alto/CY23Q2 EV Charging Event Data.xlsx', '/home/ar/Google/DTU/Courses/Advanced business analytics/ABA - Project /Datasets/Palo_Alto/CY23Q4 EV Charging Event Data.xlsx', '/home/ar/Google/DTU/Courses/Advanced business analytics/ABA - Project /Datasets/Palo_Alto/CY23Q1 EV Charging Event Data.xlsx', '/home/ar/Google/DTU/Courses/Advanced business analytics/ABA - Project /Datasets/Palo_Alto/CY22Q4 EV Charging Event Data.xlsx', '/home/ar/Google/DTU/Courses/Advanced business analytics/ABA - Project /Datasets/Palo_Alto/CY22Q3 EV Charging Event Data.xlsx', '/home/ar/Google/DTU/Courses/Advanced business analytics/ABA - Project /Datasets/Palo_Alto/CY22Q1 EV Charging Event Data.xlsx', '/home/ar/Google/DTU/Courses/Advanced business analytics/ABA - Project /Datasets/Palo_Alto/CY22Q2 EV Charging Event Data.xlsx', '/home/ar/Google/DTU/Courses/Advanced business analytics/ABA - Project /Datasets/Palo_Alto/CY23Q3 EV Ch

Read and store into single pandas dataframe all excel sheets containing complete data on Powerflex charger(that is from 2022q1 onwards) 

In [6]:
df = pd.DataFrame()

for i in new_chargers:
    data = pd.read_excel(i, sheet_name='Powerflex')
    df = pd.concat([df, data])
    
df.to_csv('only_powerflex_22_onwards', index=False)

Get data from both chargepoint format and flexpoint format, to see which ones need to be dropped, reformatted...

In [7]:
flexpoint= df.copy()
charegerpoint = together.copy()

print(flexpoint.columns)
print(charegerpoint.columns)

Index(['10-digit session UID', 'Session ID', 'Session start', 'Session end',
       'Session duration (minutes)', 'Charging duration (minutes)',
       'Session idle (minutes)', 'Estimated Completion Time', 'kWh delivered',
       'MAX kW', 'AVG kW', 'SoC Start', 'SoC End', 'User', 'Vehicle',
       'EVSE Status', 'EVSEID (PFID)', 'FSE ID', 'XB Address', 'Serial #',
       'EVSE type', 'Parking Space', 'Site', 'Site Location', 'Cost to site',
       'Cost to driver', 'Fleet', 'Session Idle (minutes)', 'Space #',
       'Session start month', 'Session start day', 'Session start hour',
       'Session start day of week', 'Session start time', 'Session end month',
       'Session end day', 'Session end hour', 'Session end day of week',
       'Session end time'],
      dtype='object')
Index(['Station Name', 'MAC Address', 'Org Name', 'Start Date',
       'Start Time Zone', 'End Date', 'End Time Zone',
       'Transaction Date (Pacific Time)', 'Total Duration (hh:mm:ss)',
       'Charging 

Merge columns 'Parking Space' and 'Site Location' to get unique charger name

In [8]:
flexpoint["Station Name"] = flexpoint["Site Location"] + flexpoint["Parking Space"]

Create two new columns containing the longitude and latitude coordinates from the adress(conatained in 'Site Location')

In [9]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import numpy
geolocator = Nominatim(user_agent="ba_project") # Initialize Nominatim API
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)# Using rate limiter to respect usage policy

# With this policy limiter it will take ages to apply the function to all rows, therefore get unique adress values and just apply fucntion to these


In [10]:

unique_values = flexpoint['Site Location'].unique()
print(unique_values)


['1213 Newell Rd, Palo Alto, CA 94303'
 '520 Webster St, Palo Alto, CA 94301'
 '445 Bryant St, Palo Alto, CA 94301'
 '250 Hamilton Ave Palo Alto, CA 94301'
 '1451 Middlefield Rd, Palo Alto, CA 94301'
 '3700 Middlefield Rd, Palo Alto, CA 94303'
 '445 Bryant St Palo Alto, CA 94301' '520 Webster St Palo Alto, CA 94301'
 nan]


In [11]:
location_dict={}#empty dictionary for mapping of adress to lat/long
for adress in unique_values:
    location =geocode(adress)
    if location:
        vals= location.latitude, location.longitude
    else:
        vals= np.nan, np.nan    
    location_dict[adress]= vals
    
print(location_dict)

{'1213 Newell Rd, Palo Alto, CA 94303': (37.4449757, -122.13907277496439), '520 Webster St, Palo Alto, CA 94301': (37.44924881470433, -122.15774617516509), '445 Bryant St, Palo Alto, CA 94301': (37.44649759802612, -122.16239223435413), '250 Hamilton Ave Palo Alto, CA 94301': (37.444848, -122.160466), '1451 Middlefield Rd, Palo Alto, CA 94301': (37.4434284, -122.14442820519255), '3700 Middlefield Rd, Palo Alto, CA 94303': (37.4221875, -122.1131623), '445 Bryant St Palo Alto, CA 94301': (37.44649759802612, -122.16239223435413), '520 Webster St Palo Alto, CA 94301': (37.44924881470433, -122.15774617516509), nan: (46.3144754, 11.0480288)}


Create two new columns in flexpoint dataframe with Latitude and Longitude information

In [12]:
# Map the latitude and longitude to new columns in the DataFrame
flexpoint['Latitude'] = flexpoint['Site Location'].map(lambda x: location_dict[x][0])
flexpoint['Longitude'] = flexpoint['Site Location'].map(lambda x: location_dict[x][1])
#error handling
flexpoint['Latitude'] = flexpoint['Site Location'].map(lambda x: location_dict.get(x, (None, None))[0])
flexpoint['Longitude'] = flexpoint['Site Location'].map(lambda x: location_dict.get(x, (None, None))[1])


Create dicctionary for new columns name assignation

In [13]:
column_mapping={
    '10-digit session UID': 'drop', 
    'Session ID': 'Plug In Event Id',
    'Session start':'Start Date',
    'Session end': 'End Date',
    'Session duration (minutes)': 'Total Duration (hh:mm:ss)', 
    'Charging duration (minutes)': 'Charging Time (hh:mm:ss)',
    'Session idle (minutes)': 'drop', 
    'Estimated Completion Time': 'drop', 
    'kWh delivered': 'Energy (kWh)',
    'MAX kW': 'drop', 
    'AVG kW': 'drop', 
    'SoC Start': 'Start SOC', 
    'SoC End': 'End SOC', 
    'User': 'User ID', 
    'Vehicle': 'drop',
    'EVSE Status': 'drop', 
    'EVSEID (PFID)': 'drop', 
    'FSE ID': 'drop', 
    'XB Address': 'drop', 
    'Serial #': 'drop',
    'EVSE type':'drop', 
    'Parking Space': 'drop', 
    'Site': 'drop', 
    #'Station Name': 'Station Name',
    'Site Location': 'Address 1', 
    'Cost to site': 'drop',
    'Cost to driver': 'drop', 
    'Fleet': 'drop', 
    'Session Idle (minutes)': 'drop', 
    'Space #': 'drop',
    'Session start month':'drop', 
    'Session start day': 'drop', 
    'Session start hour': 'drop',
    'Session start day of week': 'drop', 
    'Session start time': 'drop', 
    'Session end month': 'drop',
    'Session end day': 'drop', 
    'Session end hour': 'drop', 
    'Session end day of week': 'drop',
    'Session end time': 'drop'
}

Drop columns with value 'drop'

In [14]:
columns_to_drop = [key for key, value in column_mapping.items() if value == 'drop']
flexpoint_dropped = flexpoint.drop(columns=columns_to_drop, errors='ignore')

Rename the powerflex dataframe columns with the values from the dictionary, to make it compatible with the chargerpoint dataframe format.Additionally reorder columns in renamed flexpoint dataframe to match chargerpoint order. THis will also drop any columns left in flexpoint that are not present in chargerpoint

In [15]:
flexpoint_renamed = flexpoint_dropped.rename(columns={k: v for k, v in column_mapping.items() if v != 'drop'})

#flexpoint_renamed = flexpoint_renamed.reindex(charegerpoint.columns, axis=1)

In [16]:
# Convert minutes to timedelta
flexpoint_renamed['Total Duration (hh:mm:ss)'] = pd.to_timedelta(flexpoint_renamed['Total Duration (hh:mm:ss)'], unit='m')

# Assuming a base time at midnight of an arbitrary day
base_time = pd.Timestamp('today').normalize()

# Convert timedelta to time by adding to a base time and extracting time
flexpoint_renamed['Total Duration (hh:mm:ss)'] = (base_time + flexpoint_renamed['Total Duration (hh:mm:ss)']).dt.time

# Convert minutes to timedelta
flexpoint_renamed['Charging Time (hh:mm:ss)'] = pd.to_timedelta(flexpoint_renamed['Charging Time (hh:mm:ss)'], unit='m')

# Assuming a base time at midnight of an arbitrary day
base_time = pd.Timestamp('today').normalize()

# Convert timedelta to time by adding to a base time and extracting time
flexpoint_renamed['Charging Time (hh:mm:ss)'] = (base_time + flexpoint_renamed['Charging Time (hh:mm:ss)']).dt.time

Finally concatenate the two chargerpoint and flexpoint datframes and store complete and ready to work dataset in csv format

In [17]:
# Concatenate the two DataFrames
combined_df = pd.concat([flexpoint_renamed, charegerpoint], ignore_index=True)

# Save the combined DataFrame to a new CSV file
combined_df.to_csv('PALO_ALTO_dataset_compelete.csv', index=False)